# Visualizing Simulation Results
### Authors:
##### Justin Tung:      'https://github.com/JayTongue'
##### Matt Adam-Houser: 'https://github.com/mhouser42'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
import pickle
import networkx as nx
import ipywidgets
from ipywidgets import widgets

from run_simulation import infestation_main  # Custom function from run_simulation.py
from visualization_functions import *

---------------------------------
----------------------------------
# 1. Running and Visualizing One Simulation

Sample output from one simulation of the infestation code:

In [2]:
months_run = 50
run_mode = 'Baseline'
simulation_df = infestation_main(run_mode, months_run) # Change this to change the number of months
simulation_df

County  month 0        month 1        month 2        month 3  \
0         Cook      0.3   3.000000e-01   3.008676e-01   3.020191e-01   
1       DuPage      0.0   5.564077e-03   9.720568e-03   1.478150e-02   
2         Kane      0.1   1.046408e-01   1.118750e-01   1.220941e-01   
3         Will      0.0   4.327341e-03   7.091157e-03   1.283119e-02   
4    Winnebago      0.0   0.000000e+00   6.008489e-07   1.897513e-06   
..         ...      ...            ...            ...            ...   
97   Henderson      0.0   0.000000e+00   3.628517e-62   1.043327e-56   
98     Calhoun      0.0  1.294674e-262  2.069397e-133  3.848349e-121   
99    Schuyler      0.0   7.016762e-33   1.094564e-29   7.440999e-28   
100   Gallatin      0.0   0.000000e+00   0.000000e+00  1.244040e-249   
101       Pope      0.0   0.000000e+00   0.000000e+00   0.000000e+00   

           month 4        month 5        month 6        month 7  \
0     3.028646e-01   6.252997e-01   7.221748e-01   7.291759e-01   
1     2.068093e-02   6.073328e-02   9.036862e-02   1.123922e-01   
2     1.291713e-01   2.407059e-01   2.911570e-01   3.227666e-01   
3     1.886348e-02   4.782334e-02   9.050615e-02   1.520406e-01   
4     6.004106e-06   9.660380e-06   4.212843e-05   3.504684e-04   
..             ...            ...            ...            ...   
97    1.763411e-52   2.885687e-51   4.477558e-43   4.603900e-41   
98   7.173714e-114   3.942279e-97   4.773184e-91   1.079478e-81   
99    2.746972e-26   6.661783e-24   8.977077e-22   1.635878e-20   
100  8.781208e-225  6.886081e-214  2.770280e-202  3.442246e-194   
101   0.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00   

           month 8  ...      month 41      month 42  month 43  month 44  \
0     7.421249e-01  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
1     1.440439e-01  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
2     3.647764e-01  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
3     1.718024e-01  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
4     8.969228e-04  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
..             ...  ...           ...           ...       ...       ...   
97    1.545798e-39  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
98    2.048436e-65  ...  8.246210e-01  1.000000e+00  1.000000  1.000000   
99    2.105532e-19  ...  1.000000e+00  1.000000e+00  1.000000  1.000000   
100  6.896046e-158  ...  1.270286e-03  4.244212e-03  0.015216  0.030365   
101   0.000000e+00  ...  2.621256e-09  7.187709e-08  0.000001  0.000015   

     month 45  month 46  month 47  month 48  month 49  month 50  
0    1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
1    1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
2    1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
3    1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
4    1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
..        ...       ...       ...       ...       ...       ...  
97   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
98   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
99   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
100  0.072913  0.118661  0.199980  0.301716  0.395026  0.521233  
101  0.000096  0.000347  0.000526  0.002489  0.007983  0.010760  

[102 rows x 52 columns]

In [3]:
visual_df = make_visual_df(simulation_df)

# Make plots for each county
plt.figure(figsize=(10,10))
for county in visual_df.columns:
    plt.plot(visual_df.index, visual_df[county])
plt.xticks(ticks=range(0, months_run),
    labels=range(0, months_run))
plt.xlabel('Months')
plt.ylabel('Infestation Percentage')
plt.legend(visual_df.columns, loc=(1.01, 0), fontsize=7, ncol=2)
plt.show()

This graph is not very helpful. Since there are 102 Counties in Illinois, the resulting graph is messy and visually confusing. 

An alternative way of visualizing this data may be with a sort of heatmap instead

In [5]:
path = 'data/location'
edges = pd.read_csv(f'{path}/county_edges.csv')

CG = pickle.load(open(f'{path}/IL_graph.dat', 'rb'))
handler = pickle.load(open(f'{path}/graph_handler_counties.dat', 'rb'))
highways = pickle.load(open(f'{path}/osmnx_highways.dat', 'rb'))

In [7]:
widgets.interact(make_network_heat,
                 CG=widgets.fixed(CG),
                 handler=widgets.fixed(handler),
                 simulation_df=widgets.fixed(simulation_df),
                 month=widgets.IntSlider(min=0, max=len(simulation_df.columns)-2, step=1, layout=widgets.Layout(width='800px')),
)

interactive(children=(IntSlider(value=0, description='month', layout=Layout(width='800px'), max=50), Output())…

<function visualization_functions.make_network_heat(CG, simulation_df, handler, month: int)>

In order to better see the flow of infestations from week to week, we animated the slider:

In [8]:
play = widgets.interact(make_network_heat,
                        CG=widgets.fixed(CG),
                        handler=widgets.fixed(handler),
                        simulation_df=widgets.fixed(simulation_df),month=widgets.Play(value=0,
                                                                                      min=0,
                                                                                      max=len(simulation_df.columns) - 2,
                                                                                      step=1,
                                                                                      interval=1000,
                                                                                      disabled=False,
                                                                                     ))

interactive(children=(Play(value=0, description='month', interval=1000, max=50), Output()), _dom_classes=('wid…

Showing the infestation of the state as a whole is more helpful than each individual county:

In [ ]:
avg_df = visual_df.mean(axis=1)
plt.figure(figsize=(12,8))
plt.plot(avg_df.index, avg_df.iloc[0:, ])
plt.tick_params(labelsize=8)

--------------------------------
# 2. Running the Simulation as a Monte Carlo and Plotting the Results

With this line graph, we can run the simulation as many times as desired and compare the results.

This interactive graph shows different run modes, different experiment durations, and different numbers of simulations.

In [ ]:
widgets.interact(model_variables,
    run_mode =['Baseline', 'Poison ToH', 'Population-Based Countermeasures', 'Quarantine'],
    sims_run = (10, 100, 10),
    sim_months = (1, 30, 1),
    value='Baseline',
    disabled=False,

)

------------------------------------------------
# 3. Model the Impact of a Experimental Variable Manipulation

After running multiple simulations on each model, the resulting average trend lines can be determined for each experimental varaible.

Note that these simulations are extended to 30 months to show longer-term trends.

In [ ]:
sims_run = 100  # RIP my RAM
sim_months = 30

all_trends = {}

In [ ]:
all_trends = model_variables_avg('Baseline', sims_run, sim_months, all_trends)

In [ ]:
all_trends = model_variables_avg('Poison ToH', sims_run, sim_months, all_trends)

In [ ]:
all_trends = model_variables_avg('Population-Based Countermeasures', sims_run, sim_months, all_trends)

In [ ]:
all_trends = model_variables_avg('Quarantine', sims_run, sim_months, all_trends)

-----------------------------------------------
# 4. Compare Average Trend Lines to Determine Efficacy of Interventions

By comparing the derived trend lines, the effect of each proposed intervention can be compared.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

plt.xticks(ticks=range(0, sim_months+1),
            labels=range(0, sim_months+1))
plt.xlabel('Months')
plt.ylabel('Infestation Percentage')

for model in all_trends:
    trend_sr = all_trends[model]
    X = trend_sr.index.tolist()
    Y = trend_sr.values.tolist()
    
    ax.plot(X, Y, label=model, linewidth=3)
    ax.fill_between(X, Y, 0, alpha=0.05, interpolate=True)

plt.grid()
plt.legend(fontsize=12)
plt.show()

In [ ]:
all_trends = model_variables_avg('Poison ToH', sims_run, sim_months, all_trends)

In [ ]:
all_trends = model_variables_avg('Population-Based Countermeasures', sims_run, sim_months, all_trends)

In [ ]:
all_trends = model_variables_avg('Quarantine', sims_run, sim_months, all_trends)

-----------------------------------------------
# 4. Compare Average Trend Lines to Determine Efficacy of Interventions

By comparing the derived trend lines, the effect of each proposed intervention can be compared.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

plt.xticks(ticks=range(0, sim_months+1),
            labels=range(0, sim_months+1))
plt.xlabel('Months')
plt.ylabel('Infestation Percentage')

for model in all_trends:
    trend_sr = all_trends[model]
    X = trend_sr.index.tolist()
    Y = trend_sr.values.tolist()
    
    ax.plot(X, Y, label=model, linewidth=3)
    ax.fill_between(X, Y, 0, alpha=0.05, interpolate=True)

plt.grid()
plt.legend(fontsize=12)
plt.show()